In [2]:
import pandas as pd
import numpy as np

df_features = pd.read_parquet("data/features_d002.parquet")
df_label = pd.read_parquet("data/label_d002.parquet")
print(df_label.shape)
df_label.head()

(715680, 5)


,close,ret,trgt,bin,side
index,,,,,
2021-01-01 00:00:00+00:00,29016.23,NaN,NaN,0.0,0.0
2021-01-01 00:03:00+00:00,28965.02,NaN,NaN,0.0,0.0
2021-01-01 00:06:00+00:00,28928.06,NaN,NaN,0.0,0.0
2021-01-01 00:09:00+00:00,28851.82,NaN,NaN,0.0,0.0
2021-01-01 00:12:00+00:00,28786.75,NaN,NaN,0.0,0.0


In [3]:
meta_mask = df_label["ret"].notna().tolist()
side_mask = (df_label["side"].astype(int) != 0).tolist()

meta_label = df_label[meta_mask]["bin"].astype(np.int8).reset_index(drop=True)
meta_label.name = "label"
side_label = (
    (df_label[side_mask]["side"].astype(int) == 1)
    .astype(np.int8)
    .reset_index(drop=True)
)
side_label.name = "label"

meta_features = df_features[meta_mask].reset_index(drop=True)
side_features = df_features[side_mask].reset_index(drop=True)

assert meta_features.shape[0] == meta_label.shape[0]
assert side_features.shape[0] == side_label.shape[0]

invalid_len = 300
meta_features = meta_features.iloc[invalid_len:]
meta_label = meta_label.iloc[invalid_len:]
side_features = side_features.iloc[invalid_len:]
side_label = side_label.iloc[invalid_len:]

meta_split = int(meta_features.shape[0] * 0.8)
side_split = int(side_features.shape[0] * 0.8)

meta_x_train = meta_features.iloc[:meta_split]
meta_x_test = meta_features.iloc[meta_split:]
meta_y_train = meta_label.iloc[:meta_split]
meta_y_test = meta_label.iloc[meta_split:]

assert meta_x_train.shape[0] == meta_y_train.shape[0]
assert meta_x_test.shape[0] == meta_y_test.shape[0]

side_x_train = side_features.iloc[:side_split]
side_x_test = side_features.iloc[side_split:]
side_y_train = side_label.iloc[:side_split]
side_y_test = side_label.iloc[side_split:]

assert side_x_train.shape[0] == side_y_train.shape[0]
assert side_x_test.shape[0] == side_y_test.shape[0]

In [4]:
# meta feature selection
from featurewiz import FeatureWiz

fwiz_meta = FeatureWiz(corr_limit=0.95, verbose=0, imbalanced=True)
fwiz_meta.fit(meta_x_train, meta_y_train)
fwiz_meta.transform(meta_x_test)
fwiz_meta.features

Imported lazytransform v1.18. 

Imported featurewiz 0.5.83. Use the following syntax:
    >>> wiz = FeatureWiz(feature_engg = '', nrows=None, transform_target=True,
        		category_encoders="auto", auto_encoders='VAE', ae_options={},
        		add_missing=False, imbalanced=False, verbose=0)
    >>> X_train_selected, y_train = wiz.fit_transform(X_train, y_train)
    >>> X_test_selected = wiz.transform(X_test)
    >>> selected_features = wiz.features
    
featurewiz is given 0.9 as correlation limit...
    Skipping feature engineering since no feature_engg input...
    final list of category encoders given: ['label', 'label']
    final list of scalers given: []
Loaded input data. Shape = (427029, 1263)
#### Starting featurewiz transform for test data ####
Loaded input data. Shape = (106758, 1263)
#### Starting lazytransform for test data ####
LazyTransformer has not been fit yet. Fit it first and try again.


TypeError: object of type 'NoneType' has no len()

In [5]:
# side feature selection
fwiz_side = FeatureWiz(corr_limit=0.99, verbose=0)
fwiz_side.fit(side_x_train, side_y_train)
fwiz_side.transform(side_x_test)
fwiz_side.features

featurewiz is given 1.0 as correlation limit...
    Skipping feature engineering since no feature_engg input...
    final list of category encoders given: ['label', 'label']
    final list of scalers given: []
Loaded input data. Shape = (27514, 1263)
#### Starting featurewiz transform for test data ####
Loaded input data. Shape = (6879, 1263)
#### Starting lazytransform for test data ####
LazyTransformer has not been fit yet. Fit it first and try again.


TypeError: object of type 'NoneType' has no len()